# 🔐 Schnorr Digital Signature Implementation

A Python implementation of the Schnorr digital signature scheme for secure message authentication.

## 📋 Overview

This implementation provides a complete Schnorr signature system that allows users to:
- ✍️ **Sign messages** with cryptographic signatures
- ✅ **Verify signatures** to ensure message authenticity
- 🔒 **Prevent forgery** through discrete logarithm problem security

## 🏗️ Architecture

### 🌐 Global Parameters Generation
The system generates global public parameters `(a, p, q)` where:
- `p` is a large prime number
- `q` is a prime factor of `(p-1)`
- `a` is a generator such that `a^q ≡ 1 (mod p)`

### 🔑 Key Components

| Component | Description | Purpose |
|-----------|-------------|---------|
| `a, p, q` | Global public parameters | Shared by all users |
| `s` | Private secret key | Known only to signer |
| `v` | Public verification key | `v = a^(-s) mod p` |

## 🚀 Usage

### Basic Example
```python
# Sign a message
a, v, e, y, p = SCHNORR_SIGN("Hello World!")

# Verify the signature
is_valid = SCHNORR_VERIFY_SIGNATURE("Hello World!", a, v, e, y, p)
print(f"Signature valid: {is_valid}")  # Should print: True
```

### 📝 Message Signing Process

```python
def SCHNORR_SIGN(Message):
    # 1. Generate fresh global parameters
    a, p, q = generate_global_public_key()
    
    # 2. Generate private key
    s = random_s(q)
    
    # 3. Compute public verification key
    v = mod_negative_exp(a, s, p)  # v = a^(-s) mod p
    
    # 4. Generate random nonce
    r = random_s(q)
    
    # 5. Compute commitment
    x = mod_exp(a, r, p)  # x = a^r mod p
    
    # 6. Compute challenge
    e = int(sha256(bytes(Message + str(x), 'utf-8')).hexdigest(), 16)
    
    # 7. Compute response
    y = (r % q + (s * e) % q) % q
    
    return a, v, e, y, p
```

### ✅ Signature Verification Process

```python
def SCHNORR_VERIFY_SIGNATURE(Message, a, v, e, y, p):
    # 1. Reconstruct commitment
    x_dash = (mod_exp(a, y, p) * mod_exp(v, e, p)) % p
    
    # 2. Compute challenge hash
    e_dash = int(sha256(bytes(Message + str(x_dash), 'utf-8')).hexdigest(), 16)
    
    # 3. Verify challenge matches
    return e_dash == e
```
## 🔒 Security Features

### ✨ Cryptographic Strength
- **Discrete Logarithm Problem**: Security based on difficulty of computing `s` from `a^s mod p`
- **Random Nonce**: Each signature uses a fresh random value `r`
- **Hash Integration**: SHA-256 provides collision resistance
- **Modular Arithmetic**: All operations performed in finite field

### 🛡️ Security Properties
- ✅ **Unforgeability**: Cannot create valid signatures without private key
- ✅ **Non-repudiation**: Signer cannot deny creating a valid signature  
- ✅ **Message Integrity**: Any message modification invalidates signature
- ✅ **Randomness**: Each signature is unique due to random nonce

## 🧪 Testing

### 📊 Comprehensive Test Suite
The implementation includes extensive testing:

```python
# Test with 100,000 random messages
schnorr_random_test(100000)
# Result: Passed 100000/100000 tests ✅
```

## 📈 Performance

### ⚡ Efficiency Metrics
- **Prime Range**: 50-700 (adjustable for security/speed tradeoff)
- **Hash Function**: SHA-256 for optimal security
- **Modular Exponentiation**: Optimized square-and-multiply algorithm
- **Random Generation**: Cryptographically secure random number generation

### 🔄 Algorithm Flow
1. **Setup Phase**: Generate global parameters `(a, p, q)`
2. **Key Generation**: Create private key `s` and public key `v`
3. **Signing Phase**: Generate signature `(e, y)` for message
4. **Verification Phase**: Validate signature against message

### 🔢 Security Assumption
The security relies on the **Discrete Logarithm Problem**: Given `g^x mod p`, it's computationally infeasible to find `x`.

---
## 📖 References

- **Schnorr Signature Scheme** (Claus Schnorr, 1989)
- **Discrete Logarithm Problem** in cryptography
- **Digital Signature Standard** (DSS) specifications

---

In [2]:
import random

def is_prime_simple(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

def get_prime_factors(n):
    factors = []
    d = 2
    while n % 2 == 0:
        n //= 2
    d = 3
    while d * d <= n:
        while n % d == 0:
            factors.append(d)
            n //= d
        d += 2
    if n > 2:
        factors.append(n)

    unique_factors = []
    for f in set(factors):
        if is_prime_simple(f):
            unique_factors.append(f)
    return unique_factors


def find_a_such_that_aq_equals_1_mod_p(p, q):
    valid_a_values = []
    for a in range(2, p):
        if pow(a, q, p) == 1:
            valid_a_values.append(a)
    if valid_a_values:
        return random.choice(valid_a_values)
    return None

def generate_global_public_key():
    prime_candidates = []
    for p in range(50, 700):
        if is_prime_simple(p):
            prime_factors = get_prime_factors(p - 1)
            if prime_factors:
                prime_candidates.append(p)

    random.shuffle(prime_candidates)
    for p in prime_candidates:
        prime_factors = get_prime_factors(p - 1)
        if prime_factors:
            q = random.choice(prime_factors)
            break
    a = find_a_such_that_aq_equals_1_mod_p(p, q)
    return a, p, q
a, p ,q = generate_global_public_key()
# print(a,p,q)

In [3]:
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1

    return gcd, x, y
def mod_inverse(a, p):
    gcd, x, _ = extended_gcd(a, p)

    if gcd != 1:
        raise ValueError(f"Modular inverse does not exist for {a} mod {p}")
    return (x % p + p) % p

def mod_negative_exp(a, s, p):
    a_inverse = mod_inverse(a, p)
    result = pow(a_inverse, s, p)
    return result

In [4]:
import secrets

def random_s(q):
    return secrets.randbelow(q)
s=random_s(q)

In [5]:
def mod_exp(x, y, p):
    result = 1
    x = x % p  

    while y > 0:
        if y & 1:           # if y is odd
            result = (result * x) % p
        y >>= 1             # divide y by 2
        x = (x * x) % p     # square base

    return result


In [6]:
from hashlib import sha256
def SCHNORR_SIGN(Message):
  a, p ,q = generate_global_public_key()
  s = random_s(q)
  v=mod_negative_exp(a,s,p)
  r = random_s(q)
  x = mod_exp(a,r,p)
  # M=Message + str(x)
  e= int(sha256(bytes(Message + str(x), 'utf-8')).hexdigest(),16)
  y=(r % q + (s * e)%q) % q
  return a , v ,e , y , p

In [7]:
def SCHNORR_VERIFY_SIGNATURE(Message , a , v ,e , y,p):
  x_dash=(mod_exp(a,y,p) * mod_exp(v,e,p)) % p
  # M=Message + str(x_dash)
  e_dash= int(sha256(bytes(Message + str(x_dash), 'utf-8')).hexdigest(),16)
  return e_dash == e

In [8]:
a , v , e , y,p = SCHNORR_SIGN('HI ,SEND ME 10000 RUPPESS. VERIFY THIS SIGNATURE TO KNOW THAT THIS IS ME ')
print(SCHNORR_VERIFY_SIGNATURE('HI ,SEND ME 10000 RUPPESS. VERIFY THIS SIGNATURE TO KNOW THAT THIS IS ME ',a,v,e,y,p))

True


In [11]:
import secrets
import string

def random_message(min_len=10, max_len=100):
    length = secrets.randbelow(max_len - min_len + 1) + min_len
    alphabet = string.ascii_letters + string.digits + string.punctuation + " "
    return ''.join(secrets.choice(alphabet) for _ in range(length))

def schnorr_random_test(num_tests=60):
    passes = 0
    for i in range(num_tests):
        msg = random_message()
        a, v, e, y, p = SCHNORR_SIGN(msg)
        valid = SCHNORR_VERIFY_SIGNATURE(msg, a, v, e, y, p)
        if valid:
            passes += 1
        else:
            print(f"Test {i+1} failed! Message: {msg}")
    print(f"Passed {passes}/{num_tests} tests.")

schnorr_random_test(100000)


Passed 100000/100000 tests.
